In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dateutil
from sklearn import ensemble,metrics,linear_model,tree
from tqdm import tqdm

# Read All data

In [2]:
campData=pd.read_csv('campaign_data.csv')
coupon_item_mappingData=pd.read_csv('coupon_item_mapping.csv')
customer_demographicsData=pd.read_csv('customer_demographics.csv')
c_trans_Data=pd.read_csv('customer_transaction_data.csv')
item_dataData=pd.read_csv('item_data.csv')
trainData=pd.read_csv('train.csv')
test_QyjYwdjData=pd.read_csv('test_QyjYwdj.csv')

In [3]:
trainData['dtType']='train'
test_QyjYwdjData['dtType']='test'

allData=trainData.append(test_QyjYwdjData,sort=False)
allData.shape

(128595, 6)

In [4]:
trainData['redemption_status'].sum()

729

In [5]:
campData['start_date']=campData['start_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['end_date']=campData['end_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['duration']=campData['end_date']-campData['start_date']
campData['duration']=campData['duration'].apply(lambda x: x.days)
campData['start_Month']=campData['start_date'].apply(lambda x: x.month)
campData['end_Month']=campData['end_date'].apply(lambda x: x.month)

In [6]:
c_trans_Data['date']=c_trans_Data['date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))

In [7]:
print (max(c_trans_Data['date']),min(c_trans_Data['date']))
print (max(campData['start_date']),min(campData['start_date']))
print (max(campData['end_date']),min(campData['end_date']))

2013-12-06 00:00:00 2012-01-02 00:00:00
2013-10-21 00:00:00 2012-08-12 00:00:00
2013-12-20 00:00:00 2012-09-21 00:00:00


In [8]:
campData.sort_values('start_date')

,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month
27,26,X,2012-08-12,2012-09-21,40,8,9
26,27,Y,2012-08-25,2012-10-27,63,8,10
25,28,Y,2012-09-16,2012-11-16,61,9,11
24,29,Y,2012-10-08,2012-11-30,53,10,11
23,30,X,2012-11-19,2013-01-04,46,11,1
21,1,Y,2012-12-12,2013-01-18,37,12,1
22,2,Y,2012-12-17,2013-01-18,32,12,1
18,3,Y,2012-12-22,2013-02-16,56,12,2
20,4,Y,2013-01-07,2013-02-08,32,1,2
19,5,Y,2013-01-12,2013-02-15,34,1,2


In [9]:
for j,k,l in tqdm(zip(campData['start_date'],campData['end_date'],campData['campaign_id'])):
    c_trans_Data['campaign_'+str(l).zfill(2)]=c_trans_Data['date'].apply(lambda x:1 if ((x >= j) & (x <= k)) else 0)
#     if (c_trans_Data['date'][1324561] >= j) & (c_trans_Data['date'][1324561] <= k):
#         print (j,k)

28it [01:30,  3.24s/it]


In [10]:
campCol=['campaign_24', 'campaign_25',
       'campaign_20', 'campaign_23', 'campaign_21', 'campaign_22',
       'campaign_18', 'campaign_19', 'campaign_17', 'campaign_16',
       'campaign_13', 'campaign_11', 'campaign_12', 'campaign_10',
       'campaign_09', 'campaign_08', 'campaign_07', 'campaign_06',
       'campaign_03', 'campaign_05', 'campaign_04', 'campaign_01',
       'campaign_02', 'campaign_30', 'campaign_29', 'campaign_28',
       'campaign_27', 'campaign_26']

In [11]:
campTran=c_trans_Data[campCol]

campTran.iloc[1324544][campTran.iloc[1324544]==1].index

campTran['campParticipated']=campTran.apply(lambda x: x[x==1].index.values, axis = 1)

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
c_trans_Data['campParticipated']=campTran['campParticipated']
c_trans_Data['filt']=c_trans_Data['campParticipated'].apply(lambda x: len(x))
for j in campCol:
    del c_trans_Data[j]

In [13]:
c_trans_Data.shape

(1324566, 9)

In [14]:
c_trans_Data=c_trans_Data[c_trans_Data['filt']!=0]
c_trans_Data['campaign_id']=c_trans_Data['campParticipated'].apply(lambda x: int(x[0].split('_')[1]))
del c_trans_Data['filt']
del c_trans_Data['campParticipated']

In [15]:
mergedData=pd.merge(c_trans_Data,campData,on='campaign_id',how='left')

In [16]:
mergedData['occurenceofPurch']=mergedData['date']-mergedData['start_date']
mergedData['occurenceofPurch']=mergedData['occurenceofPurch'].apply(lambda x: x.days)

In [17]:
mergedData.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month,occurenceofPurch
0,2012-09-01,1497,9281,4,71.24,0.00,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
1,2012-09-01,1497,18919,2,284.25,-103.30,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
2,2012-09-01,1497,19849,1,44.17,0.00,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
3,2012-09-01,1497,23126,2,106.86,-34.91,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
4,2012-09-01,1497,30874,1,88.69,0.00,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7


In [18]:
bohotData=pd.merge(mergedData,coupon_item_mappingData,on='item_id',how='left')
# bohotData=pd.merge(bohotData,campData,on='item_id',how='left')
bohotData.shape

(1761105, 16)

In [35]:
def getFeatureGener(df,indCol,fCol):
    groupedMerge=df.groupby(indCol)
    c_data={}
    for c_id,tempT in  groupedMerge:
        c_data[c_id]={}
        c_data[c_id][indCol+fCol+'_max']=max(tempT[fCol])
        c_data[c_id][indCol+fCol+'_min']=min(tempT[fCol])
        c_data[c_id][indCol+fCol+'_mean']=np.mean(tempT[fCol])
        c_data[c_id][indCol+fCol+'_75perc']=np.percentile(tempT[fCol],75)
        c_data[c_id][indCol+fCol+'_25perc']=np.percentile(tempT[fCol],25)
    tempDF=pd.DataFrame(c_data).transpose()
    tempDF.columns=[indCol+fCol+'_max',indCol+fCol+'_min',indCol+fCol+'_mean',indCol+fCol+'_75perc',indCol+fCol+'_25perc']
    return tempDF,tempDF.index

In [36]:
quantityCData,indexC=getFeatureGener(mergedData,'customer_id','quantity')
selling_priceCData,indexC=getFeatureGener(mergedData,'customer_id','selling_price')
other_discountCData,indexC=getFeatureGener(mergedData,'customer_id','other_discount')
coupon_discountCData,indexC=getFeatureGener(mergedData,'customer_id','coupon_discount')
occurenceofPurchCData,indexC=getFeatureGener(mergedData,'customer_id','occurenceofPurch')

In [37]:
featureCustomerData=pd.concat([quantityCData,selling_priceCData,other_discountCData,coupon_discountCData,occurenceofPurchCData],axis=1)

In [38]:
featureCustomerData=featureCustomerData.reset_index()
featureCustomerData.columns=['customer_id']+list(featureCustomerData.columns[1:])

In [39]:
featureCustomerData.head()

,customer_id,customer_idquantity_max,customer_idquantity_min,customer_idquantity_mean,customer_idquantity_75perc,customer_idquantity_25perc,customer_idselling_price_max,customer_idselling_price_min,customer_idselling_price_mean,customer_idselling_price_75perc,...,customer_idcoupon_discount_max,customer_idcoupon_discount_min,customer_idcoupon_discount_mean,customer_idcoupon_discount_75perc,customer_idcoupon_discount_25perc,customer_idoccurenceofPurch_max,customer_idoccurenceofPurch_min,customer_idoccurenceofPurch_mean,customer_idoccurenceofPurch_75perc,customer_idoccurenceofPurch_25perc
0,1,1.0,1.0,5.0,1.153968,1.0,64.5625,106.500,445.25,93.758000,...,0.0,0.0,0.0,-2.630746,-106.86,9.0,26.0,43.0,17.819048,0.0
1,2,1.0,1.0,4.0,1.123404,1.0,56.6400,117.190,640.80,102.176043,...,0.0,0.0,0.0,-0.151574,-35.62,4.0,25.0,38.0,15.255319,0.0
2,3,1.0,2.0,6949.0,18.264286,1.0,53.4300,120.750,862.00,103.992905,...,0.0,0.0,0.0,-3.509357,-142.48,3.0,25.0,39.0,13.107143,0.0
3,4,1.0,1.0,4.0,1.187500,1.0,70.8800,142.120,1330.05,153.771000,...,0.0,0.0,0.0,0.000000,0.00,5.0,15.0,44.0,12.312500,0.0
4,5,1.0,1.0,14638.0,178.203065,1.0,60.2000,166.615,1485.35,141.278927,...,0.0,0.0,0.0,-0.174004,-71.24,6.0,24.0,46.0,15.383142,0.0


In [40]:
quantityIData,indexI=getFeatureGener(mergedData,'item_id','quantity')
selling_priceIData,indexI=getFeatureGener(mergedData,'item_id','selling_price')
other_discountIData,indexI=getFeatureGener(mergedData,'item_id','other_discount')
coupon_discountIData,indexI=getFeatureGener(mergedData,'item_id','coupon_discount')
occurenceofPurchIData,indexI=getFeatureGener(mergedData,'item_id','occurenceofPurch')

In [41]:
itemFeatureData=pd.concat([quantityIData,selling_priceIData,other_discountIData,coupon_discountIData,occurenceofPurchIData],axis=1)

In [42]:
itemFeatureData=itemFeatureData.reset_index()
itemFeatureData.columns=['item_id']+list(itemFeatureData.columns[1:])

In [43]:
bohotData.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month,occurenceofPurch,coupon_id
0,2012-09-01,1497,9281,4,71.24,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,NaN
1,2012-09-01,1497,18919,2,284.25,-103.3,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,1043.0
2,2012-09-01,1497,19849,1,44.17,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,24.0
3,2012-09-01,1497,19849,1,44.17,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,8.0
4,2012-09-01,1497,19849,1,44.17,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,33.0


In [44]:
quantityCPData,indexCP=getFeatureGener(bohotData,'coupon_id','quantity')
selling_priceCPData,indexCP=getFeatureGener(bohotData,'coupon_id','selling_price')
other_discountCPData,indexCP=getFeatureGener(bohotData,'coupon_id','other_discount')
coupon_discountCPData,indexCP=getFeatureGener(bohotData,'coupon_id','coupon_discount')
occurenceofPurchCPData,indexCP=getFeatureGener(bohotData,'coupon_id','occurenceofPurch')

In [45]:
couponFeatureData=pd.concat([quantityCPData,selling_priceCPData,other_discountCPData,coupon_discountCPData,occurenceofPurchCPData],axis=1)

In [46]:
couponFeatureData=couponFeatureData.reset_index()
couponFeatureData.columns=['coupon_id']+list(couponFeatureData.columns[1:])

In [47]:
couponFeatureData.head(2)

,coupon_id,coupon_idquantity_max,coupon_idquantity_min,coupon_idquantity_mean,coupon_idquantity_75perc,coupon_idquantity_25perc,coupon_idselling_price_max,coupon_idselling_price_min,coupon_idselling_price_mean,coupon_idselling_price_75perc,...,coupon_idcoupon_discount_max,coupon_idcoupon_discount_min,coupon_idcoupon_discount_mean,coupon_idcoupon_discount_75perc,coupon_idcoupon_discount_25perc,coupon_idoccurenceofPurch_max,coupon_idoccurenceofPurch_min,coupon_idoccurenceofPurch_mean,coupon_idoccurenceofPurch_75perc,coupon_idoccurenceofPurch_25perc
0,1.0,1.0,1.0,4.0,1.239478,1.0,71.24,113.63,362.26,103.692830,...,0.0,0.0,0.0,-2.109274,-142.48,6.0,24.0,47.0,15.699565,0.0
1,2.0,1.0,1.0,4.0,1.320755,1.0,89.05,110.07,356.20,126.734528,...,0.0,0.0,0.0,0.000000,0.00,6.0,28.0,44.0,17.415094,0.0


# Merge traindata with Campdata

In [49]:
allData.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType
0,1,13,27,1053,0.0,train
1,2,13,116,48,0.0,train


In [156]:
X1=pd.merge(allData,featureCustomerData,on='customer_id',how='left')
X1=pd.merge(X1,couponFeatureData,on='coupon_id',how='left')
X1=pd.merge(X1,campData,on='campaign_id',how='left')
X1=pd.merge(X1,customer_demographicsData,on='customer_id',how='left')

In [157]:
X1.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,customer_idquantity_max,customer_idquantity_min,customer_idquantity_mean,customer_idquantity_75perc,...,end_date,duration,start_Month,end_Month,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0.0,train,1.0,2.0,12444.0,288.855932,...,2013-07-05,47,5,7,46-55,NaN,0.0,1,NaN,5.0
1,2,13,116,48,0.0,train,1.0,1.0,6.0,1.376068,...,2013-07-05,47,5,7,36-45,Married,0.0,2,NaN,3.0
2,6,9,635,205,0.0,train,1.0,1.0,10.0,1.347759,...,2013-04-12,32,3,4,46-55,Married,0.0,2,NaN,7.0
3,7,13,644,1050,0.0,train,1.0,1.0,5.0,1.299270,...,2013-07-05,47,5,7,NaN,NaN,NaN,NaN,NaN,NaN
4,9,8,1017,1489,0.0,train,1.0,2.0,15420.0,260.419355,...,2013-04-05,48,2,4,46-55,Married,0.0,2,NaN,3.0


In [158]:
X1.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'dtType', 'customer_idquantity_max', 'customer_idquantity_min',
       'customer_idquantity_mean', 'customer_idquantity_75perc',
       'customer_idquantity_25perc', 'customer_idselling_price_max',
       'customer_idselling_price_min', 'customer_idselling_price_mean',
       'customer_idselling_price_75perc', 'customer_idselling_price_25perc',
       'customer_idother_discount_max', 'customer_idother_discount_min',
       'customer_idother_discount_mean', 'customer_idother_discount_75perc',
       'customer_idother_discount_25perc', 'customer_idcoupon_discount_max',
       'customer_idcoupon_discount_min', 'customer_idcoupon_discount_mean',
       'customer_idcoupon_discount_75perc',
       'customer_idcoupon_discount_25perc', 'customer_idoccurenceofPurch_max',
       'customer_idoccurenceofPurch_min', 'customer_idoccurenceofPurch_mean',
       'customer_idoccurenceofPurch_75perc',
       'customer_ido

In [159]:
X5=X1

In [160]:
X5['marital_status']=X5['marital_status'].fillna('Other')
X5['rented']=X5['rented'].fillna(0)
X5['no_of_children']=X5['no_of_children'].fillna('0')
X5['age_range']=X5['age_range'].fillna('Other')
X5['family_size']=X5['family_size'].fillna('9999')
X5['income_bracket']=X5['income_bracket'].fillna(9999)

In [161]:
X5.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'dtType', 'customer_idquantity_max', 'customer_idquantity_min',
       'customer_idquantity_mean', 'customer_idquantity_75perc',
       'customer_idquantity_25perc', 'customer_idselling_price_max',
       'customer_idselling_price_min', 'customer_idselling_price_mean',
       'customer_idselling_price_75perc', 'customer_idselling_price_25perc',
       'customer_idother_discount_max', 'customer_idother_discount_min',
       'customer_idother_discount_mean', 'customer_idother_discount_75perc',
       'customer_idother_discount_25perc', 'customer_idcoupon_discount_max',
       'customer_idcoupon_discount_min', 'customer_idcoupon_discount_mean',
       'customer_idcoupon_discount_75perc',
       'customer_idcoupon_discount_25perc', 'customer_idoccurenceofPurch_max',
       'customer_idoccurenceofPurch_min', 'customer_idoccurenceofPurch_mean',
       'customer_idoccurenceofPurch_75perc',
       'customer_ido

In [162]:
from sklearn import preprocessing,model_selection

In [163]:
llP=preprocessing.LabelEncoder()
# X5[jj]=llP.fit_transform(X5['campaign_type'])

In [164]:
categoColumns=['age_range', 'marital_status', 'rented', 'family_size',#'category',# 'brand', 'brand_type',
       'no_of_children', 'income_bracket','campaign_type']
X5[categoColumns].head(2)

,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_type
0,46-55,Other,0.0,1,0,5.0,X
1,36-45,Married,0.0,2,0,3.0,X


In [165]:
X5['income_bracket']=X5['income_bracket'].map(str)
X5['rented']=X5['rented'].map(str)

In [166]:
pd.unique(X5['family_size'])

array(['1', '2', '9999', '3', '4', '5+'], dtype=object)

In [167]:

catData=pd.get_dummies(X5[categoColumns],prefix_sep='_',)
# for jj in categoColumns:
#     print (jj)
#     llP=preprocessing.LabelEncoder()
#     X5[jj]=llP.fit_transform(X5[jj])

In [168]:
newtData=pd.concat([X5,catData],sort=False,axis=1)
# newtData=X5

In [169]:
newtData.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'dtType', 'customer_idquantity_max', 'customer_idquantity_min',
       'customer_idquantity_mean', 'customer_idquantity_75perc',
       ...
       'income_bracket_3.0', 'income_bracket_4.0', 'income_bracket_5.0',
       'income_bracket_6.0', 'income_bracket_7.0', 'income_bracket_8.0',
       'income_bracket_9.0', 'income_bracket_9999.0', 'campaign_type_X',
       'campaign_type_Y'],
      dtype='object', length=105)

In [170]:
XVar=list(newtData.columns)
XVar.remove('id')
XVar.remove('redemption_status')
XVar.remove('dtType')
XVar.remove('start_date')
XVar.remove('end_date')
YVar='redemption_status'

In [171]:
for j in categoColumns:#['campaign_id', 'coupon_id', 'customer_id']:
    XVar.remove(j)

In [172]:
trainPart=newtData[newtData['dtType']=='train']
testPart=newtData[newtData['dtType']=='test']
trainPart.shape,testPart.shape

((78369, 105), (50226, 105))

In [173]:
from sklearn import model_selection,metrics
import xgboost as xgb

In [174]:
# 	if feature_names is not None:
# 		create_feature_map(feature_names)
# 		model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
# 		importance = model.get_fscore(fmap='xgb.fmap')
# 		importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
# 		imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
# 		imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
# 		imp_df.to_csv("imp_feat.txt", index=False)

# 	pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
# 	pred_test_y2 = model.predict(xgb.DMatrix(test_X2), ntree_limit=model.best_ntree_limit)

# 	loss = 0
# 	if test_y is not None:
# 		loss = metrics.roc_auc_score(test_y, pred_test_y)
# 		return pred_test_y, loss, pred_test_y2
# 	else:
# 		return pred_test_y, loss, pred_test_y2

## Train test Split Train, Val, Test set

In [175]:
trainX,testHX,trainY,testHY=model_selection.train_test_split(trainPart[XVar],trainPart[YVar],test_size=.2,stratify =trainPart[YVar])
trainX.shape,testHX.shape,trainY.shape,testHY.shape

((62695, 93), (15674, 93), (62695,), (15674,))

In [176]:
trainX1,valX1,trainY1,valY1=model_selection.train_test_split(trainX,trainY,test_size=.2,stratify =trainY)
trainX1.shape,valX1.shape,trainY1.shape,valY1.shape

((50156, 93), (12539, 93), (50156,), (12539,))

In [177]:
XVar

['campaign_id',
 'coupon_id',
 'customer_id',
 'customer_idquantity_max',
 'customer_idquantity_min',
 'customer_idquantity_mean',
 'customer_idquantity_75perc',
 'customer_idquantity_25perc',
 'customer_idselling_price_max',
 'customer_idselling_price_min',
 'customer_idselling_price_mean',
 'customer_idselling_price_75perc',
 'customer_idselling_price_25perc',
 'customer_idother_discount_max',
 'customer_idother_discount_min',
 'customer_idother_discount_mean',
 'customer_idother_discount_75perc',
 'customer_idother_discount_25perc',
 'customer_idcoupon_discount_max',
 'customer_idcoupon_discount_min',
 'customer_idcoupon_discount_mean',
 'customer_idcoupon_discount_75perc',
 'customer_idcoupon_discount_25perc',
 'customer_idoccurenceofPurch_max',
 'customer_idoccurenceofPurch_min',
 'customer_idoccurenceofPurch_mean',
 'customer_idoccurenceofPurch_75perc',
 'customer_idoccurenceofPurch_25perc',
 'coupon_idquantity_max',
 'coupon_idquantity_min',
 'coupon_idquantity_mean',
 'coupon_i

In [178]:
trainX.columns

Index(['campaign_id', 'coupon_id', 'customer_id', 'customer_idquantity_max',
       'customer_idquantity_min', 'customer_idquantity_mean',
       'customer_idquantity_75perc', 'customer_idquantity_25perc',
       'customer_idselling_price_max', 'customer_idselling_price_min',
       'customer_idselling_price_mean', 'customer_idselling_price_75perc',
       'customer_idselling_price_25perc', 'customer_idother_discount_max',
       'customer_idother_discount_min', 'customer_idother_discount_mean',
       'customer_idother_discount_75perc', 'customer_idother_discount_25perc',
       'customer_idcoupon_discount_max', 'customer_idcoupon_discount_min',
       'customer_idcoupon_discount_mean', 'customer_idcoupon_discount_75perc',
       'customer_idcoupon_discount_25perc', 'customer_idoccurenceofPurch_max',
       'customer_idoccurenceofPurch_min', 'customer_idoccurenceofPurch_mean',
       'customer_idoccurenceofPurch_75perc',
       'customer_idoccurenceofPurch_25perc', 'coupon_idquantity_

In [179]:
# def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, feature_names=None, seed_val=0, rounds=500, dep=8, eta=0.05):
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = 'auc'
params["eta"] = .05
params["subsample"] = 0.7
params["min_child_weight"] = 1
params["colsample_bytree"] = 0.7
params["max_depth"] = 8

params["silent"] = 1
params["seed"] = 200
#params["max_delta_step"] = 2
#params["gamma"] = 0.5
num_rounds = 500

plst = list(params.items())
xgtrain = xgb.DMatrix(trainX1, label=trainY1)

# if test_y is not None:
xgtest = xgb.DMatrix(valX1, label=valY1)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=20)
# else:
#     xgtest = xgb.DMatrix(test_X)
#     model = xgb.train(plst, xgtrain, num_rounds)

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-auc:0.69471	test-auc:0.679629
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[20]	train-auc:0.932749	test-auc:0.923776
[40]	train-auc:0.966188	test-auc:0.942657
[60]	train-auc:0.982915	test-auc:0.948315
[80]	train-auc:0.988662	test-auc:0.952516
[100]	train-auc:0.992551	test-auc:0.95581
[120]	train-auc:0.995222	test-auc:0.956141
[140]	train-auc:0.996556	test-auc:0.956884
[160]	train-auc:0.997769	test-auc:0.956749
[180]	train-auc:0.998661	test-auc:0.955987
[200]	train-auc:0.999265	test-auc:0.954751
[220]	train-auc:0.999647	test-auc:0.95486
[240]	train-auc:0.999844	test-auc:0.953911
[260]	train-auc:0.999928	test-auc:0.95366
Stopping. Best iteration:
[164]	train-auc:0.997946	test-auc:0.957157



In [180]:
t1=model.predict(xgtrain)
t2=model.predict(xgtest)
xgTest2 = xgb.DMatrix(testHX)
t3=model.predict(xgTest2)

# trainScore=modelR.predict_proba(trainX)
# testScore=modelR.predict_proba(valX1)
# test2Score=modelR.predict_proba(testHX)

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY1,t1),metrics.roc_auc_score(valY1,t2),metrics.roc_auc_score(testHY,t3)

train 583.0 vs 452.29650962078495 validation 117.0 vs 79.05900800281006 test 146.0 vs 95.493761224021


(0.9999423032241961, 0.9530884686254192, 0.9464264289696739)

In [98]:
plst = list(params.items())
xgtrain = xgb.DMatrix(trainPart[XVar],label=trainPart[YVar])

In [101]:
model2 = xgb.train(plst, xgtrain, num_rounds)

In [102]:
xgTestData = xgb.DMatrix(testPart[XVar])

In [103]:
t3Score=model.predict(xgTestData)

In [181]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:t3Score})
firstSub2.to_csv('xgbSub5.csv',index=False)

In [ ]:
import lightgbm  as lgb

In [ ]:
modelL=lgb.LGBMClassifier().fit(trainX1,trainY1)

In [ ]:
t1=modelL.predict(trainX)
t2=modelL.predict(valX1)
t3=modelL.predict(testHX)
trainScore=modelL.predict_proba(trainX)[:,1]
testScore=modelL.predict_proba(valX1)[:,1]
test2Score=modelL.predict_proba(testHX)[:,1]

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY,trainScore),metrics.roc_auc_score(valY1,testScore),metrics.roc_auc_score(testHY,test2Score)

In [ ]:
params={'base_score':0.9,
        'booster':'gbtree', 
        'colsample_bylevel':1,
       'colsample_bytree':1, 
        'gamma':0, 
        'learning_rate':0.1, 
        'max_delta_step':0,
       'max_depth':3, 
        'min_child_weight':1, 
        'n_estimators':100,
        'eval_metric': ['logloss','auc'],
       'n_jobs':1,  'objective':'binary:logistic', 'random_state':0,
       'reg_alpha':0, 'reg_lambda':1, 'scale_pos_weight':1, 'subsample':1,
       'early_stopping_rounds' : 50,
       'stratified':True }

In [ ]:
num_round = 1000

dtrain = xgb.DMatrix(trainX1,trainY1)
# dtrain.set_group(Xg)

dvalid = xgb.DMatrix(valX1, valY1)
# dvalid.set_group(Xgt)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
bst = xgb.train(params, dtrain, num_round, evals = watchlist,
    early_stopping_rounds=50,verbose_eval=50,maximize=True)


In [ ]:
dtest = xgb.DMatrix(testHX)

In [ ]:
testScoreP=bst.predict(dtest)

In [ ]:
metrics.roc_auc_score(testHY,testScoreP)

In [ ]:
dtestOrg = xgb.DMatrix(testPart[XVar])
testScoreP2=bst.predict(dtestOrg)

In [ ]:
testScoreP2

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:testScoreP2})
firstSub2.to_csv('firstSub9.csv',index=False)

## LGB model

In [ ]:
modelL.fit(trainPart[XVar],trainPart[YVar])

In [ ]:
lgbScore=modelL.predict_proba(testPart[XVar])[:,1]
lgbScore2=modelL.predict(testPart[XVar])

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:lgbScore})
firstSub2.to_csv('lgbSub3.csv',index=False)

## XGB model

In [ ]:
modelR=xgb.XGBClassifier().fit(trainPart[XVar],trainPart[YVar])
xgbScore=modelL.predict_proba(testPart[XVar])[:,1]
xgbScore2=modelL.predict(testPart[XVar])

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:xgbScore})
firstSub2.to_csv('xgbSub3.csv',index=False)